In [58]:
import pandas as pd

import requests
import os

from tqdm.auto import tqdm  # for notebooks

tqdm.pandas()

from warnings import simplefilter

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [59]:
file_path = "replays.parquet"
if os.path.exists(file_path):
    disk = pd.read_parquet(file_path)
    if "id" in disk.columns:
        disk = disk.set_index("id")


fetch_new = True
if fetch_new:
    apiUrl = "https://api.bar-rts.com/replays?limit=1000&preset=team&hasBots=true"

    json = requests.get(apiUrl, headers={"User-Agent": "tetrisface"}).json()

    data = json["data"]

    api = pd.DataFrame.from_records(data).set_index("id")

    if disk is None:
        games = api
    else:
        games = pd.concat(
            [disk, api.loc[api.index.difference(disk.index)]],
            verify_integrity=True,
            axis=0,
        )
    games.startTime = pd.to_datetime(games.startTime)

    games.to_parquet(file_path)
else:
    games = disk

In [60]:
pd.options.mode.copy_on_write = True
import time


def is_raptors(row):
    for team in row:
        for ai in team["AIs"]:
            if ai["shortName"] == "RaptorsAI":
                return True
    return False


def is_draw(row):
    results = []
    for team in row:
        results.append(team["winningTeam"])
    return len(team) <= 1 or all(
        x == results[0] for x in [team["winningTeam"] for team in row]
    )


def winners(row):
    _winners = []
    for team in row:
        if team["winningTeam"] is True:
            if len(team["Players"]) > 0:
                _winners.extend([player["name"] for player in team["Players"]])
            elif len(team["AIs"]) > 0:
                _winners.extend([ai["shortName"] for ai in team["AIs"]])
    return _winners


def players(row):
    _players = []
    for team in row:
        _players.extend([player["name"] for player in team["Players"]])
    return _players


games["raptors"] = games["AllyTeams"].apply(is_raptors)
games["draw"] = games["AllyTeams"].apply(is_draw)
games["winners"] = games["AllyTeams"].apply(winners)
games["players"] = games["AllyTeams"].apply(players)

In [61]:
numerical_columns = [
    "ai_incomemultiplier",
    "air_rework",
    "allowpausegameplay",
    "allowuserwidgets",
    "april1",
    "april1extra",
    "assistdronesair",
    "assistdronesbuildpowermultiplier",
    "assistdronescount",
    "capturebonus",
    "captureradius",
    "capturetime",
    "commanderbuildersbuildpower",
    "commanderbuildersrange",
    "coop",
    "critters",
    "debugcommands",
    "decapspeed",
    "defaultdecals",
    "disable_fogofwar",
    "disablemapdamage",
    "dominationscore",
    "dominationscoretime",
    "easter_egg_hunt",
    "easteregghunt",
    "emprework",
    "energyperpoint",
    "experimentalextraunits",
    "experimentalimprovedtransports",
    "experimentallegionfaction",
    "experimentalmassoverride",
    "experimentalnoaircollisions",
    "experimentalrebalancet2energy",
    "experimentalrebalancet2labs",
    "experimentalrebalancet2metalextractors",
    "experimentalrebalancewreckstandarization",
    "experimentalreversegear",
    "experimentalxpgain",
    "faction_limiter",
    "ffa_wreckage",
    "fixedallies",
    "lategame_rebalance",
    "limitscore",
    "map_atmosphere",
    "map_waterislava",
    "map_waterlevel",
    "maxunits",
    "metalperpoint",
    "multiplier_builddistance",
    "multiplier_buildpower",
    "multiplier_buildtimecost",
    "multiplier_energyconversion",
    "multiplier_energycost",
    "multiplier_energyproduction",
    "multiplier_losrange",
    "multiplier_maxdamage",
    "multiplier_maxvelocity",
    "multiplier_metalcost",
    "multiplier_metalextraction",
    "multiplier_radarrange",
    "multiplier_resourceincome",
    "multiplier_shieldpower",
    "multiplier_turnrate",
    "multiplier_weapondamage",
    "multiplier_weaponrange",
    "norush",
    "norushtimer",
    "numberofcontrolpoints",
    "proposed_unit_reworks",
    "ranked_game",
    "raptor_endless",
    "raptor_firstwavesboost",
    "raptor_graceperiodmult",
    "raptor_queentimemult",
    "raptor_spawncountmult",
    "raptor_spawntimemult",
    "releasecandidates",
    "ruins_civilian_disable",
    "ruins_only_t1",
    "scav_bosstimemult",
    "scav_endless",
    "scav_graceperiodmult",
    "scav_spawncountmult",
    "scav_spawntimemult",
    "scoremode_chess_adduptime",
    "scoremode_chess_spawnsperphase",
    "scoremode_chess_unbalanced",
    "scoremode_chess",
    "shareddynamicalliancevictory",
    "skyshift",
    "startenergy",
    "startenergystorage",
    "startmetal",
    "startmetalstorage",
    "starttime",
    "teamffa_start_boxes_shuffle",
    "tugofwarmodifier",
    "unified_maxslope",
    "unit_restrictions_noair",
    "unit_restrictions_noconverters",
    "unit_restrictions_noendgamelrpc",
    "unit_restrictions_noextractors",
    "unit_restrictions_nolrpc",
    "unit_restrictions_nonukes",
    "unit_restrictions_notacnukes",
    "unit_restrictions_notech2",
    "unit_restrictions_notech3",
    "usemapconfig",
    "usemexconfig",
]
string_columns = [
    "assistdronesenabled",
    "commanderbuildersenabled",
    "deathmode",
    "experimentalshields",
    "experimentalstandardgravity",
    "lootboxes_density",
    "lootboxes",
    "map_tidal",
    "raptor_difficulty",
    "raptor_raptorstart",
    "ruins_density",
    "ruins",
    "scav_difficulty",
    "scav_scavstart",
    "scoremode",
    "teamcolors_anonymous_mode",
    "teamcolors_icon_dev_mode",
    "transportenemy",
    "tweakdefs",
    "tweakdefs1",
    "tweakdefs2",
    "tweakdefs3",
    "tweakdefs4",
    "tweakdefs5",
    "tweakdefs6",
    "tweakdefs7",
    "tweakdefs8",
    "tweakdefs9",
    "tweakunits",
    "tweakunits1",
    "tweakunits2",
    "tweakunits3",
    "tweakunits4",
    "tweakunits5",
    "tweakunits6",
    "tweakunits7",
    "tweakunits8",
    "tweakunits9",
]


def cast_frame(_df):

    for col in string_columns:
        _df[string_columns] = _df[string_columns].fillna("")

    _df = _df.astype({col: str for col in string_columns}, errors="raise")

    for col in numerical_columns:
        _df[numerical_columns] = _df[numerical_columns].fillna(0)

    for col in numerical_columns:
        _df[col] = pd.to_numeric(
            _df[col],
            downcast="integer",
        )

    return _df

In [62]:
raptor_games = games[
    games["raptors"]
    # & ~df_root_expanded["draw"]
]  # draws might be good to exclude

raptor_games["fetch_success"] = None


def api_game_settings(row):
    time.sleep(1)
    if row is not None and row.name is not None:
        url = f"https://api.bar-rts.com/replays/{row.name}"
        response = requests.get(url, headers={"User-Agent": "tetrisface"})
        if response.status_code == 200:
            game_settings = response.json().get("gameSettings")
            for key, value in game_settings.items():
                # Add new column to DataFrame if the column doesn't exist
                if key not in raptor_games.columns:
                    raptor_games[key] = None
                # Update DataFrame with fetched data
                row[key] = value
            row["fetch_success"] = True
            return row
    print(f"Failed to fetch data from {url}")
    row["fetch_success"] = False
    return row


# load in disk
gamesettings_path = "replays_gamesettings.parquet"
if os.path.exists(gamesettings_path):
    disk = pd.read_parquet(gamesettings_path)
    raptor_games.loc[disk.index, disk.columns] = disk

isnull = raptor_games[
    raptor_games.fetch_success.isnull() | (raptor_games.fetch_success == False)
]
to_fetch = isnull.head(200)
print(f"fetching {len(to_fetch)} of {len(isnull)} missing games")

# fetch new
df_raptors_api = to_fetch.progress_apply(
    api_game_settings,
    axis=1,
)
raptor_games.loc[df_raptors_api.index, df_raptors_api.columns] = cast_frame(
    df_raptors_api
)

if len(raptor_games.loc[raptor_games.fetch_success == False]) > 0:
    print(f"failed to fetch {len(raptor_games[~raptor_games.fetch_success])} games")

raptor_games = cast_frame(raptor_games)

fetching 0 of 0 missing games


1it [00:01,  1.00s/it]


In [63]:
raptor_games.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2261 entries, 0cb40a66f0eb04a7025bc929bf299f35 to b71c0f6634c9c60b2f23eb0b179f7f75
Data columns (total 156 columns):
 #    Column                                    Dtype              
---   ------                                    -----              
 0    startTime                                 datetime64[ns, UTC]
 1    durationMs                                int64              
 2    Map                                       object             
 3    AllyTeams                                 object             
 4    raptors                                   bool               
 5    draw                                      bool               
 6    winners                                   object             
 7    players                                   object             
 8    fetch_success                             object             
 9    transportenemy                            object             
 10   tweakunits3     

In [64]:
# store
raptor_games[raptor_games["fetch_success"].notnull()].to_parquet(gamesettings_path)

In [65]:
from nuttyb import hp_multiplier, main, coms, meganuke_149, wind_restrict_149, gamesettings

raptor_games["nuttyb_main"] = raptor_games.apply(
    lambda row: all(
        [
            tweak["value"] == row[tweak["location"]]
            for tweak in main
            if tweak["version"] == "1.48"
        ]
    )
    or all(
        [
            tweak["value"] == row[tweak["location"]]
            for tweak in main
            if tweak["version"] == "1.49"
        ]
    ),
    axis=1,
)


def nuttyb_difficulty(row):
    for _def in hp_multiplier:
        if 'values' in _def:
            if row[_def["location"]] in _def["values"]:
                return _def["name"]
        else:
            if row[_def["location"]] == _def["value"]:
                return _def["name"]
    return None


raptor_games["nuttyb_hp"] = raptor_games.apply(nuttyb_difficulty, axis=1)


possible_tweaks = ['tweakunits', 'tweakdefs'] +[f'tweakunits{i}' for i in range(1, 10)] + [f'tweakdefs{i}' for i in range(1, 10)]
all_allowed_tweaks = []
for setting_dict in [setting_dict for setting_dict in [*main, *hp_multiplier, *coms, meganuke_149, wind_restrict_149]]:
    if 'values' in setting_dict:
        all_allowed_tweaks.extend(setting_dict['values'])
    else:
        all_allowed_tweaks.append(setting_dict['value'])

def is_default_nuttyb(row):
    if row["nuttyb_main"] \
        and row["nuttyb_hp"] is not None and len(row["nuttyb_hp"]) > 0 \
        and row[possible_tweaks][row[possible_tweaks].astype(bool)].isin(all_allowed_tweaks).all():
        #     and row['NuttyB Mode'] is not None and len(row['NuttyB Mode']) > 0
        return True
    return False

raptor_games["default_nuttyb"] = raptor_games.apply(is_default_nuttyb, axis=1)


def raptor_diff(row):
    if row["default_nuttyb"]:
        return f"NuttyB Default {row['nuttyb_hp']}"
    elif row["nuttyb_main"]:
        if row["nuttyb_hp"]:
            return f"NuttyB Main & HP {row["nuttyb_hp"]}"
        else:
            return f"NuttyB Main & Vanilla {row["raptor_difficulty"]}"
    elif row["nuttyb_hp"]:
        return f"Vanilla & NuttyB HP {row["nuttyb_hp"]}"
    else:
        return f"Vanilla {row["raptor_difficulty"]}"


raptor_games["Difficulty"] = pd.Categorical(
    raptor_games.apply(
        raptor_diff,
        axis=1),
        [
            "Vanilla veryeasy",
            "Vanilla easy",
            "Vanilla normal",
            "Vanilla hard",
            "Vanilla veryhard",
            "Vanilla epic",
            "Vanilla & NuttyB HP Epic",
            "Vanilla & NuttyB HP Epic+",
            "Vanilla & NuttyB HP Epic++",
            "Vanilla & NuttyB HP Epicer+",
            "Vanilla & NuttyB HP Epicest",
            "NuttyB Default Epic",
            "NuttyB Default Epic+",
            "NuttyB Default Epic++",
            "NuttyB Default Epicer+",
            "NuttyB Default Epicest",
            "NuttyB Main & HP Epic",
            "NuttyB Main & HP Epic+",
            "NuttyB Main & HP Epic++",
            "NuttyB Main & HP Epicer+",
            "NuttyB Main & HP Epicest",
            "NuttyB Main & Vanilla normal",
            "NuttyB Main & Vanilla hard",
            "NuttyB Main & Vanilla veryhard",
            "NuttyB Main & Vanilla epic",
        ],
        ordered=True,
)

def nuttyb_mode(row):
    for name, settings in gamesettings.items():
        if all([row[setting] == value for setting, value in settings.items()]):
            return name
    return None
raptor_games["NuttyB Mode"] = raptor_games.apply(nuttyb_mode, axis=1)

assert (
    len(raptor_games[raptor_games["Difficulty"].isna()]) == 0
), f'missing difficulties for {len(raptor_games[raptor_games["Difficulty"].isna()])} {raptor_games[raptor_games["Difficulty"].isna()][['nuttyb_main', "nuttyb_hp",'raptor_difficulty', 'Difficulty', 'NuttyB Mode']]}'

In [74]:
raptor_players = raptor_games.explode("players")
raptor_players.rename(columns={"players": "player"}, inplace=True)


raptor_players["won"] = raptor_players.apply(
    lambda row: row.player in row.winners, axis=1
)

clearers = raptor_players[raptor_players["won"] & raptor_players["default_nuttyb"]]
all_groups = pd.DataFrame()
for (group_mode, group_diff), group_df in clearers.groupby(
    ["NuttyB Mode", "Difficulty"], observed=True
):
    group_players = (
        group_df.groupby("player")
        .size()
        .sort_values(ascending=False)
        #  .head(20)
        .to_frame()
        .reset_index()
    )
    group_players["NuttyB Mode"] = group_mode
    group_players["Difficulty"] = group_diff
    group_players.rename(columns={0: "Clears"}, inplace=True)
    all_groups = pd.concat(
        [
            group_players[
                [
                    "NuttyB Mode",
                    "Difficulty",
                    "player",
                    "Clears",
                ]
            ],
            all_groups,
        ],
        axis=1,
    )
all_groups.columns = map(str.capitalize, all_groups.columns)
display(all_groups.fillna("").replace(1.0, 1))

,Nuttyb mode,Difficulty,Player,Clears
0,rush,NuttyB Default Epic+,myriari,2
1,rush,NuttyB Default Epic+,ArmCommander,1
2,rush,NuttyB Default Epic+,974625813,1
3,rush,NuttyB Default Epic+,ChronoBR,1
4,rush,NuttyB Default Epic+,Finray,1
5,rush,NuttyB Default Epic+,FirstCore,1
6,rush,NuttyB Default Epic+,Backbash,1
7,rush,NuttyB Default Epic+,GenJermain,1
8,rush,NuttyB Default Epic+,HotPotato,1
9,rush,NuttyB Default Epic+,JunoMiNha1,1


In [67]:
print(f'nuttyb main {len(raptor_games[raptor_games["nuttyb_main"]])}')
print(
    f'nuttyb main + hp {len(raptor_games[raptor_games["nuttyb_main"] & raptor_games["nuttyb_hp"]])}'
)
print(f'default nuttyb {len(raptor_games[raptor_games["default_nuttyb"]])}')
test_df = raptor_games[raptor_games["Difficulty"] == "NuttyB Default Epic++"]
print(raptor_games[raptor_games["Difficulty"] == "NuttyB Default Epic++"])

nuttyb main 530
nuttyb main + hp 470
default nuttyb 156
                                                 startTime  durationMs  \
id                                                                       
46990c66b597eee2b2d25216620d652b 2024-04-02 23:48:22+00:00     3904100   
fa710c66f73c71dfa33f4c03c074067d 2024-04-02 21:00:42+00:00     2873933   

                                                                                Map  \
id                                                                                    
46990c66b597eee2b2d25216620d652b  {'fileName': 'full_metal_plate_1.7', 'scriptNa...   
fa710c66f73c71dfa33f4c03c074067d  {'fileName': 'full_metal_plate_1.7', 'scriptNa...   

                                                                          AllyTeams  \
id                                                                                    
46990c66b597eee2b2d25216620d652b  [{'AIs': [{'shortName': 'RaptorsAI'}], 'Player...   
fa710c66f73c71dfa33f4c03c074067d  [{'

In [81]:
import csv


# player_game_mask = raptor_games["players"].apply(lambda x: 'tetrisface' in x)
player_game_mask = False

total_unique = raptor_players.player.nunique()

print(
    f'{len(raptor_games[player_game_mask] if player_game_mask else raptor_games)} games and {total_unique} player names between {raptor_players["startTime"].min().date()} and {raptor_players["startTime"].max().date()}'
)
# raptor_players["Difficulty"] =
agg_total = raptor_players.groupby(["Difficulty"], observed=True).agg(
    {"player": "nunique"}
)
agg_won = (
    raptor_players[raptor_players["won"]]
    .groupby(["Difficulty"], observed=True)
    .agg({"player": "nunique"})
)
game_modes = agg_total.join(agg_won, lsuffix="s that tried", rsuffix="s that won")
game_modes["% won"] = (
    game_modes["players that won"] / game_modes["players that tried"] * 100
)
game_modes["% tried of total"] = game_modes["players that tried"] / total_unique * 100
game_modes["% won of total"] = game_modes["players that won"] / total_unique * 100
game_modes.fillna(0, inplace=True)
game_modes["players that won"] = game_modes["players that won"].astype(int)
game_modes.to_csv("game_modes_stats.csv", quoting=csv.QUOTE_ALL)
display(
    game_modes.style.format(
        {"% won": "{:.0f}%", "% tried of total": "{:.0f}%", "% won of total": "{:.0f}%"}
    ).set_table_styles(
        [{"selector": ".row_heading", "props": [("text-align", "left")]}]
    )
)

2261 games and 2547 player names between 2024-03-15 and 2024-04-04


,players that tried,players that won,% won,% tried of total,% won of total
Difficulty,,,,,
Vanilla veryeasy,41,16,39%,2%,1%
Vanilla easy,77,45,58%,3%,2%
Vanilla normal,811,330,41%,32%,13%
Vanilla hard,304,96,32%,12%,4%
Vanilla veryhard,545,91,17%,21%,4%
Vanilla epic,692,209,30%,27%,8%
Vanilla & NuttyB HP Epic,45,20,44%,2%,1%
Vanilla & NuttyB HP Epic+,585,268,46%,23%,11%
Vanilla & NuttyB HP Epic++,20,0,0%,1%,0%


In [69]:
from math import log2
from collections import Counter


def ID3_entropies(data_df):
    """
    https://gist.github.com/whitehaven/bbd408edca38de93637635b52d2bba89

    Takes pandas.DataFrame and returns a series with all non-index schemas' entropies calculated.

    It supports non-binary field types by calculating average entropy. Result series starts with the most productive decision level.
    """

    def entropy_for_field(field):
        entropy = 0
        field_entry_count = len(field)

        # get count of unique
        field_counter = Counter(field)

        # E( Si/S * E(pi*log2(pi)) )
        for trait, count in field_counter.items():
            p_T = count / field_entry_count
            p_F = (field_entry_count - count) / field_entry_count

            if p_T == 0 or p_F == 0:
                entropy = 0
                break
            # Si/S * E(pi*log2(pi))
            entropy += (
                count / field_entry_count * (-(p_T * log2(p_T)) - (p_F * log2(p_F)))
            )
        return entropy

    data_df_entropy = {}
    for field in data_df:
        entropy_this_field = entropy_for_field(data_df[field])
        data_df_entropy[field] = entropy_this_field

    data_df_entropy_se = pd.Series(data_df_entropy)
    data_df_entropy_se.sort_values(inplace=True, ascending=False)
    return data_df_entropy_se


df_entropy = ID3_entropies(
    raptor_games.loc[:, raptor_games.columns.isin(numerical_columns)]
).to_frame("entropy")

In [70]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    df_stats = (
        raptor_games.loc[:, raptor_games.columns.isin(numerical_columns)]
        .agg(["mean", "median", "min", "max", "std", "skew", "kurt"])
        .T.join(df_entropy, how="outer")
        .sort_values(by="entropy", ascending=False)
    )
df_stats.to_csv("raptor_gamesettings_stats.csv", quoting=csv.QUOTE_ALL)
with pd.option_context(
    "display.float_format", "{:.0f}".format, "display.max_rows", 500
):
    display(
        df_stats.style.format(
            {
                "median": "{:.1f}",
                "min": "{:.1f}",
                "max": "{:.0f}",
                "mean": "{:.2f}",
                "std": "{:.2f}",
                "skew": "{:.2f}",
                "kurt": "{:.2f}",
                "entropy": "{:.2f}",
            }
        ).set_table_styles(
            [{"selector": ".row_heading", "props": [("text-align", "left")]}]
        )
    )

,mean,median,min,max,std,skew,kurt,entropy
releasecandidates,0.54,1.0,0.0,1,0.50,-0.16,-1.98,1.00
experimentalrebalancewreckstandarization,0.58,1.0,0.0,1,0.49,-0.33,-1.89,0.98
experimentalextraunits,0.60,1.0,0.0,1,0.49,-0.43,-1.82,0.97
experimentallegionfaction,0.63,1.0,0.0,1,0.48,-0.53,-1.72,0.95
commanderbuildersbuildpower,630.21,400.0,100.0,1000,286.96,0.48,-1.73,0.93
maxunits,4920.52,2000.0,1000.0,10000,3742.49,0.58,-1.61,0.89
unit_restrictions_noextractors,0.30,0.0,0.0,1,0.46,0.86,-1.26,0.88
startenergy,4784.38,1000.0,1000.0,10000,4242.38,0.35,-1.80,0.88
startmetal,4736.39,1500.0,1000.0,10000,4215.08,0.37,-1.78,0.86
raptor_spawncountmult,1.94,1.0,1.0,5,1.12,0.83,-0.22,0.86


In [71]:
# df_raptors = df_raptors.explode("players")
# df_raptors.rename(columns={"players": "player"}, inplace=True)
# df_raptors["won"] = df_raptors.apply(lambda row: row.player in row.winners, axis=1)

# df_raptors["week"] = pd.to_datetime(df_raptors["startTime"]).dt.month
# df = df_raptors.groupby(["player", "week"]).agg({"won": "mean", "id": "count"})
# df = df.rename(columns={"won": "winrate", "id": "replays"})

# df = df[df.replays > 4]

# rank_sum_column = "rank_sum"
# df[rank_sum_column] = (df.winrate.rank() * 0.1 + df.replays.rank() * 0.9) / len(df)
# df.sort_values(rank_sum_column, inplace=True, ascending=False)

# df[rank_sum_column] = round(df[rank_sum_column], 2).astype(str)

# df.winrate = df.winrate * 100

In [72]:
# pd.set_option("display.max_rows", 500)
# pd.options.display.float_format = "{:.0f}%".format

# for week, df_week in df.groupby("week"):
#     # df_week = df_week[df_week["replays"] > 4].reset_index()
#     df_week = df_week.reset_index()
#     df_week.index = np.arange(1, len(df_week) + 1)
#     print(df_week.head(20))
#     print()